In [12]:
import serial
import os
from IPython.display import clear_output

base = "data_max_integration_time/"

NO_LED = "NO_LED/"
UV = "UV/"
VIS = "VIS/"
IR = "IR/"

n_samples = 2


In [13]:
arduino = serial.Serial(port='/dev/cu.usbserial-1420', baudrate=115200, timeout=.1)

# Data collection setup

batch_number = 29

In [14]:
import time

NO_LED_batch_path = os.path.join(base, NO_LED, f"{str(batch_number)}.csv")
UV_batch_path = os.path.join(base, UV, f"{str(batch_number)}.csv")
VIS_batch_path = os.path.join(base, VIS, f"{str(batch_number)}.csv")
IR_batch_path = os.path.join(base, IR, f"{str(batch_number)}.csv")

batches = [
    {"path": NO_LED_batch_path, "command": "get"},
    {"path": UV_batch_path, "command": "get:UV"}, 
    {"path": IR_batch_path, "command": "get:IR"}, 
    {"path": VIS_batch_path, "command": "get:VIS"}, 
]

if os.path.exists(UV_batch_path) or os.path.exists(VIS_batch_path) or os.path.exists(IR_batch_path) or os.path.exists(NO_LED_batch_path):
    print("Batch already exists. Exiting.")
else:
    for batch in batches:
        with open(batch["path"], 'w') as f:
            for x in range(n_samples):
                clear_output(wait=True)
                line = ""
                arduino.write(bytes(batch["command"], 'utf-8'))
                while True:
                    # Wait until arduino sends data
                    if arduino.in_waiting > 0:
                        line += arduino.readline().decode()
                        # Log process
                        print(f"Reading {batch['command']}... {x+1}/{n_samples} samples")
                        break
                f.write(line)
                time.sleep(3) # Wait between samples to prevent overheating

Reading get:VIS... 2/2 samples


In [15]:
glucose_concentration = 7.1

for batch in batches:
    with open(batch["path"], 'a') as f:
        f.write(f"glucose_concentration:{glucose_concentration}\n")

#### Collecting samples in time

In [9]:
import time

final_base = "final_evaluation_data/"
num_samples = 2
sampling_time = 30

UV_final_path = os.path.join(final_base, "UV.csv")
VIS_final_path = os.path.join(final_base, "VIS.csv")
IR_final_path = os.path.join(final_base, "IR.csv")

batches = [
    {"path": UV_final_path, "command": "get:UV"}, 
    {"path": IR_final_path, "command": "get:IR"}, 
    {"path": VIS_final_path, "command": "get:VIS"}, 
]

for curr_time in range(sampling_time):
    print(f"Sampling time: {curr_time+1}/{sampling_time} minutes")
    
    for batch in batches:
        with open(batch["path"], 'a') as f:
            for x in range(num_samples):
                # clear_output(wait=True)
                line = ""
                arduino.write(bytes(batch["command"], 'utf-8'))
                while True:
                    # Wait until arduino sends data
                    if arduino.in_waiting > 0:
                        line += arduino.readline().decode()
                        # Log process
                        print(f"Reading {batch['command']}... {x+1}/{n_samples} samples")
                        break
                f.write(line)
                time.sleep(3) # Wait between samples to prevent overheating
                
    print("Waiting 1 minute for next sample")
    time.sleep(60) # Wait 1 minute between samples

Sampling time: 1/30 minutes
Reading get:UV... 1/2 samples
Reading get:UV... 2/2 samples
Reading get:IR... 1/2 samples
Reading get:IR... 2/2 samples
Reading get:VIS... 1/2 samples
Reading get:VIS... 2/2 samples
Waiting 1 minute for next sample
Sampling time: 2/30 minutes
Reading get:UV... 1/2 samples
Reading get:UV... 2/2 samples
Reading get:IR... 1/2 samples
Reading get:IR... 2/2 samples
Reading get:VIS... 1/2 samples
Reading get:VIS... 2/2 samples
Waiting 1 minute for next sample


KeyboardInterrupt: 